In [1]:
import os
import groq

from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from typing import List
from pydantic import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

In [2]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [3]:
t = convert_to_openai_function(Tagging)
tools = {"type": "function",
         "function": t}
tools

{'type': 'function',
 'function': {'name': 'Tagging',
  'description': 'Tag the piece of text with particular info.',
  'parameters': {'type': 'object',
   'properties': {'sentiment': {'type': 'string'},
    'language': {'type': 'string'}},
   'required': ['sentiment', 'language']}}}

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

model = ChatGroq(temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [5]:
model_with_functions = model.bind(tools=[tools])
tagging_chain = prompt | model_with_functions

In [6]:
tagging_chain.invoke({"input": "I am figuring out Langchain"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ksq6', 'function': {'arguments': '{"language":"English","sentiment":"neutral"}', 'name': 'Tagging'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.157630511, 'completion_tokens': 99, 'prompt_time': 0.262217746, 'prompt_tokens': 1221, 'queue_time': None, 'total_time': 0.41984825699999995, 'total_tokens': 1320}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7fc9c543-0b91-4623-ad9b-a9bb744c50a7-0', tool_calls=[{'name': 'Tagging', 'args': {'language': 'English', 'sentiment': 'neutral'}, 'id': 'call_ksq6'}])

In [26]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain | model_with_functions | JsonOutputFunctionsParser()
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_408r', 'function': {'arguments': '{"sentiment":"negative","language":"italian"}', 'name': 'Tagging'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.156075637, 'completion_tokens': 99, 'prompt_time': 0.413110686, 'prompt_tokens': 1222, 'queue_time': None, 'total_time': 0.569186323, 'total_tokens': 1321}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fe9c8f5c-d718-4a1e-9f78-1ba3f031c31f-0', tool_calls=[{'name': 'Tagging', 'args': {'sentiment': 'negative', 'language': 'italian'}, 'id': 'call_408r'}])

In [33]:
# Extraction

from typing import Optional

class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")


In [37]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [40]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
convert_pydantic_to_openai_function(Information)

c:\Users\Dhanuka\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'$defs': {'Person': {'description': 'Information about a person.',
    'properties': {'name': {'description': "person's name", 'type': 'string'},
     'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
      'description': "person's age"}},
    'required': ['name', 'age'],
    'type': 'object'}},
  'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

In [49]:
extraction_function = [convert_pydantic_to_openai_function(Information)]
extraction_func = {"type": "function",
         "function": extraction_function}
extraction_func

{'type': 'function',
 'function': [{'name': 'Information',
   'description': 'Information to extract.',
   'parameters': {'$defs': {'Person': {'description': 'Information about a person.',
      'properties': {'name': {'description': "person's name",
        'type': 'string'},
       'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
        'description': "person's age"}},
      'required': ['name', 'age'],
      'type': 'object'}},
    'properties': {'people': {'description': 'List of info about people',
      'items': {'description': 'Information about a person.',
       'properties': {'name': {'description': "person's name",
         'type': 'string'},
        'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
         'description': "person's age"}},
       'required': ['name', 'age'],
       'type': 'object'},
      'type': 'array'}},
    'required': ['people'],
    'type': 'object'}}]}

In [50]:
extraction_model = model.bind(tools=[extraction_func])

In [51]:
extraction_model.invoke("Joe is 30, his mom is Martha")

BadRequestError: Error code: 400 - {'error': {'message': "'tools.0.function' : value must be an object", 'type': 'invalid_request_error'}}

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [48]:
extraction_chain = prompt | extraction_model
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

BadRequestError: Error code: 400 - {'error': {'message': "'tools.0.function' : value must be an object", 'type': 'invalid_request_error'}}

In [56]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

BadRequestError: Error code: 400 - {'error': {'message': "'tools.0.function' : value must be an object", 'type': 'invalid_request_error'}}

In [55]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser # Looks for perticular key
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

BadRequestError: Error code: 400 - {'error': {'message': "'tools.0.function' : value must be an object", 'type': 'invalid_request_error'}}

In [57]:
# Real World Example

from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()


In [58]:
doc = documents[0]
page_content = doc.page_content[:10000]
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [59]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [68]:
overview_tagging_function = [
    convert_to_openai_function(Overview)
]
overview_tagging_function


[{'name': 'Overview',
  'description': 'Overview of a section of text.',
  'parameters': {'type': 'object',
   'properties': {'summary': {'type': 'string'},
    'language': {'type': 'string'},
    'keywords': {'type': 'string'}},
   'required': ['summary', 'language', 'keywords']}}]

In [69]:
overview_tagging_func = {"type": "function",
         "function": overview_tagging_function}
overview_tagging_func

{'type': 'function',
 'function': [{'name': 'Overview',
   'description': 'Overview of a section of text.',
   'parameters': {'type': 'object',
    'properties': {'summary': {'type': 'string'},
     'language': {'type': 'string'},
     'keywords': {'type': 'string'}},
    'required': ['summary', 'language', 'keywords']}}]}

In [71]:
tagging_model = model.bind(tools=[overview_tagging_func])
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()
tagging_chain.invoke({"input": page_content})

BadRequestError: Error code: 400 - {'error': {'message': "'tools.0.function' : value must be an object", 'type': 'invalid_request_error'}}